In [ ]:
%%writefile .env
#OPENAI_API_KEY="sk-proj-"
#TAVILY_API_KEY=tvly-dev-

Writing .env


In [2]:
pip uninstall -y openai-agents

In [3]:
pip install -q openai-agents==0.2.2 python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 2.4 MB/s eta 0:00:00


In [4]:
pip install python-dotenv langchain-openai==0.2.1

In [ ]:
pip install --upgrade pydantic

In [6]:
import os
from openai import OpenAI
from IPython.display import display, Markdown

# This will be used to load the API key from the .env file
from dotenv import load_dotenv
load_dotenv()

# Get the OpenAI API keys from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# Let's configure the OpenAI Client using our key
openai_client = OpenAI(api_key = openai_api_key)
print("OpenAI client successfully configured.")

# Let's view the first few characters in the key
print(openai_api_key[:5])
print(tavily_api_key[:5])

OpenAI client successfully configured.
sk-pr
tvly-


In [7]:
def print_markdown(text):
    """Displays text as Markdown in Jupyter."""
    display(Markdown(text))

In [8]:
# Import the OpenAI API client
from openai import OpenAI

# Import the Agent class to create and manage AI agents
from agents import Agent

# Define the instructions for the fact-checker AI Agent
fact_checker_instructions = """
Context:
You are a fact-checker who verifies the accuracy of statements.

Instructions:
When given a statement, carefully analyze its factual accuracy using your knowledge.

Input:
You will receive a statement that requires fact-checking.

Output:
Respond with:
1. A verdict prefix: either "✅ TRUE:" or "❌ FALSE:"
2. A brief, one-sentence explanation justifying your conclusion
"""

In [9]:
# Create a new agent called "Fact Checker"
fact_checker_agent = Agent(name = "Fact Checker",   # Name of the agent
                           instructions = fact_checker_instructions, # The rules and behavior for the agent
                           model = "gpt-4o-mini") # The AI model (LLM) to use

# Print a confirmation message that the agent was created
print(f"Agent '{fact_checker_agent.name}' created successfully!")

Agent 'Fact Checker' created successfully!


In [10]:
# Import the Runner class, which is used to run an agent and get its output
from agents import Runner

# A statement we want the Fact Checker agent to verify
statement = "The Great Wall of China is not visible from space with the naked eye."

# Display the statement we're going to check (in markdown format for nicer formatting)
print_markdown(f"Asking the Fact Checker to verify: '{statement}'")

# Run the Fact Checker agent on the input statement
# 'await' is used because running the agent is an asynchronous operation (it might take time)
response = await Runner.run(
    starting_agent = fact_checker_agent,  # The agent we created earlier
    input = statement                 # The statement we want it to fact-check
)

# Display the agent's response
print_markdown("\n🤖 Agent's Response:\n")
print_markdown(response.final_output)    # Shows the final verdict and explanation

Asking the Fact Checker to verify: 'The Great Wall of China is not visible from space with the naked eye.'


🤖 Agent's Response:


✅ TRUE: The Great Wall of China is generally not visible from space without aid because it is narrow and blends in with the natural terrain, contrary to popular belief.

In [9]:
from agents import Agent, Runner, SQLiteSession

# Define the role and instructions for the AI agent
market_researcher_instructions = """
Context:
You are a market research assistant helping analyze companies, industries, and competitors.

Instructions:
When given a question, provide a short factual answer based on your knowledge.

Output:
Start with a verdict prefix: either "✅ FACT:" or "❌ UNKNOWN:"
Follow with a concise one-sentence explanation.
"""

# Create an instance of the Agent
market_researcher_agent = Agent(name = "Market Researcher",
                                instructions = market_researcher_instructions,
                                model = "gpt-4.1-mini")

In [17]:
# Let's give our first question to the AI agent
q1 = "What is the market share of Tesla in the US EV market?"

# Display the user’s question in Markdown format
print_markdown(f"You: '{q1}'")

# Run the agent with the first question (no memory means each query is independent)
resp1 = await Runner.run(starting_agent = market_researcher_agent, input = q1)

# Display the agent’s response
print_markdown(f"🤖 Agent:\n{resp1.final_output}")

You: 'What is the market share of Tesla in the US EV market?'

🤖 Agent:
✅ FACT: Tesla holds approximately 60-70% of the US electric vehicle (EV) market share as of early 2024.

In [18]:
# Second question — depends on previous context
# Follow-up question that refers to the previous answer
q2 = "How does that compare to last year?"

# Display the follow-up question
print_markdown(f"\nYou: '{q2}'")

# Run the agent again — since there’s no memory, it does not recall the first question/answer
resp2 = await Runner.run(starting_agent = market_researcher_agent, input = q2)

# Display the agent’s response (will fail to connect it to the first question)
print_markdown(f"🤖 Agent:\n{resp2.final_output}")


You: 'How does that compare to last year?'

🤖 Agent:
❌ UNKNOWN: You have not provided the specific data or context needed to compare this year to last year.

In [10]:
# Create a session instance
# SQLite-based implementation of session storage.
# This implementation stores conversation history in a SQLite database.

session = SQLiteSession("conversation")

# First interaction
user_input1 = "What is the market share of Tesla in the US EV market?"
response1 = await Runner.run(
    starting_agent = market_researcher_agent,
    input = user_input1,
    session = session,
)

print_markdown(f"🤖 Agent:\n{response1.final_output}")

🤖 Agent:
✅ FACT: Tesla holds approximately 60-70% of the US electric vehicle (EV) market share as of early 2024.

In [20]:
# Second interaction with history
user_input2 = "How does that compare to last year?"

response2 = await Runner.run(
    starting_agent = market_researcher_agent,
    input = user_input2,
    session = session,
)

print_markdown(f"🤖 Agent:\n{response2.final_output}")

🤖 Agent:
✅ FACT: Tesla's US EV market share has slightly decreased from about 70-75% in 2023 to around 65-70% in early 2024 due to increasing competition.

In [10]:
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict  # Import TypedDict for type hinting
from agents import function_tool

In [11]:
# Define a TypedDict for the expected parameters for the Tavily search function
# A TypedDict is like a blueprint for a dictionary in Python
# It tells Python exactly what keys the dictionary should have and what type of values go with each key.

class TavilySearchParams(TypedDict):
    query: str         # The search query string
    max_results: int   # The maximum number of results to return

In [12]:
import requests

@function_tool
def tavily_search(params: TavilySearchParams) -> str:
    """
    Calls the Tavily API and returns a string summary of top search results.

    Args:
        params (TavilySearchParams): Dictionary with 'query' (str) and 'max_results' (int).

    Returns:
        str: A formatted string summarizing the top search results, or an error message.
    """

    # The web address (endpoint) for sending search requests to Tavily ( Tavily API endpoint)
    url = "https://api.tavily.com/search"

    # Tell the API that we’re sending JSON data
    headers = {"Content-Type": "application/json"}

    # What we’re sending to the API:
    # Our secret API key (so Tavily knows it's us)
    # The search text (query)
    # How many results we want (defaults to 2 if not given)

    payload = {
        "api_key": tavily_api_key,  # Use the Tavily API key from environment
        "query": params["query"],   # The search query from params
        "max_results": params.get("max_results", 2),  # Use max_results from params, default to 2 if not provided
    }

    # Send the search request to Tavily (POST means we’re sending data)
    response = requests.post(url, json = payload, headers = headers)

    # Check if the search worked (200 = OK)
    if response.status_code == 200:  # If the request was successful
        results = response.json().get("results", [])  # Extract the 'results' list from the response JSON
        print_markdown(f"### 🤖 Agent’s Answer\n{results}")
        # Build a summary string with each result's title and content, numbered
        summary = "\n".join([f"{i+1}. {r['title']}: {r['content']}" for i, r in enumerate(results)])
        return summary if summary else "No relevant results found."  # Return summary or fallback message
    else:
        return f"Tavily API error: {response.status_code}"  # Return error message with status code if request failed

In [13]:
from agents import SQLiteSession
session = SQLiteSession("live_researcher_practice")

In [14]:
from agents import Agent

live_researcher_agent = Agent(name = "Live Market Researcher",
                             instructions = """
CONTEXT:
You are a world-class market research assistant with access to real-time web search via the tavily_search tool.

INSTRUCTION:
- Analyze the user's question and determine if recent or real-time information is needed.
- If the question involves recent events, news, or product info, always call tavily_search.
- Summarize search results clearly and concisely, do not copy-paste.
- Always start your answer with: "🔍 According to a web search …"

INPUT:
You will receive a conversation history and the latest user question. Use the full context to inform your response.

OUTPUT:
Provide a clear, well-structured answer that references the search results when appropriate. If you use tavily_search, integrate the findings into your text summary only with 2-3 paragraphs.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search])

print("✅ Agent created with Tavily tool.")

✅ Agent created with Tavily tool.


In [39]:
from agents import Runner

# First question
q1 = "What are people saying about the new GPT-5 Model?"

print_markdown(f"**User:** {q1}")

run1 = await Runner.run(
    starting_agent = live_researcher_agent,
    input = q1,
    session = session,
)

print_markdown(f"### 🤖 Agent’s Answer\n{run1.final_output}")


**User:** What are people saying about the new GPT-5 Model?

### 🤖 Agent’s Answer
🔍 According to a web search, the new GPT-5 model is being received with a mix of excitement and thoughtful critique. People highlight its key innovation as a unified system that smartly routes requests to different sub-models based on the complexity and type of the task. This real-time router can adjust reasoning depth and effort, allowing GPT-5 to efficiently handle both straightforward and challenging problems without user intervention.

The model is praised as OpenAI's strongest yet for coding tasks, notably for complex front-end code generation and debugging of large codebases. Users appreciate that GPT-5 "just does stuff" by deciding internally how much "thinking" to apply, which simplifies the user experience compared to earlier versions where users had to choose among several models manually.

However, some more advanced users note that the model can be somewhat arbitrary in determining which problems require deeper reasoning, occasionally leading to longer processing times or inconsistent reasoning effort. Despite this, the consensus is that GPT-5 significantly reduces cognitive load for users, streamlines interaction with AI by hiding model selection complexities, and marks a major step forward in AI capability and usability.

In [15]:
from agents import Runner

session = SQLiteSession("live_researcher_practice")

prod_memory = []

q1 = "What do reviwers say about the new Tesla CyberTruck"

run1 = await Runner.run(
    starting_agent = live_researcher_agent,
    input = q1,
    session = session,
)

print_markdown(f"### 🤖 Agent’s Answer\n{run1.final_output}")

### 🤖 Agent’s Answer
[{'url': 'https://cleantechnica.com/2024/05/01/tesla-cybertruck-review-3000-miles-in-10-days/', 'title': 'Tesla Cybertruck Review — 3,000 Miles in 10 Days - CleanTechnica', 'content': '# Tesla Cybertruck Review — 3,000 Miles in 10 Days **Driving** — The Cybertruck’s “scale weight” is listed at 6,669 pounds on the vehicle registration. The Cybertruck drives better than my 2016 Model S — I was amazed by this fact! The Cybertruck accelerates like my Model S, even when passing trucks going up mountains on steep inclines. Perhaps it is the way we view the Cybertruck — as a utility vehicle that does “city stuff” but also goes off road, hauls dirty cargo (like sod and topsoil), and makes dump runs. **Range and Charging** — The Cybertruck when fully charged showed 318 miles of range.', 'score': 0.8452414, 'raw_content': None}, {'url': 'https://www.motortrend.com/news/2024-tesla-cybertruck-pros-cons-review', 'title': '2024 Tesla Cybertruck Pros and Cons Review: The No-Hype True ...', 'content': '# 2024 Tesla Cybertruck TOTY Review: The No-Hype, No-Spin True Truck Test *This review was conducted as part of our 2025 Truck of the Year (TOTY) testing, where each vehicle is evaluated on our six key criteria: efficiency, design, safety, engineering excellence, value, and performance of intended function. ### Tesla Cybertruck Beast is Both Quicker and Slower than Tesla’s Claims in MotorTrend Testing ### 2024 Tesla Cybertruck Dual Motor Real-World Range Test: Beat by Ford and Rivian ### 2024 Tesla Cybertruck Dual Motor Foundation Series First Test: You’re All Wrong ### 2024 Tesla Cybertruck Dual Motor 0–60 MPH and ¼-Mile Tested: Can It Cyberdust Other Electric Trucks?', 'score': 0.8036831, 'raw_content': None}, {'url': 'https://www.caranddriver.com/tesla/cybertruck', 'title': '2025 Tesla Cybertruck Review, Pricing, and Specs - Car and Driver', 'content': "The Tesla Cybertruck is a giant pickup with silly-quick acceleration and a design that blows minds—and it's unexpectedly capable and nice to drive.", 'score': 0.779339, 'raw_content': None}, {'url': 'https://www.reddit.com/r/teslamotors/comments/1hxo8fw/10000_mile_cybertruck_review/', 'title': '10,000 Mile Cybertruck Review : r/teslamotors - Reddit', 'content': '10,000 Mile Cybertruck Review : r/teslamotors Skip to main content10,000 Mile Cybertruck Review : r/teslamotors Image 1: r/teslamotors icon Go to teslamotors Image 3: r/teslamotors iconr/teslamotorsImage 4: bluecheckImage 5: bluecheck The original and largest Tesla community on Reddit! 10,000 Mile Cybertruck Review 10,000 mile Tesla Cybertruck review insights Best reviews for Tesla Cybertruck New to Reddit? Top Posts *   Reddit reReddit: Top posts of January 9, 2025 * * * *   Reddit reReddit: Top posts of January 2025 * * * *   Reddit reReddit: Top posts of 2025 * * * *   Reddit Meta *   Games  *   Gaming News & Discussion *   Other Games *   Action Movies & Series *   Animated Movies & Series *   About Reddit *   Best of Reddit', 'score': 0.7783298, 'raw_content': None}, {'url': 'https://electrek.co/2024/06/08/tesla-cybertruck-review-incredible-tech-packaged-weirdest-way/', 'title': 'Tesla Cybertruck review: incredible tech packaged in the weirdest way', 'content': 'It is a very novel vehicle right now, but Tesla is ramping up production and plans to make hundreds of thousands of Cybertrucks per year. I wouldn’t be surprised if the vehicle program were born from a Tesla designer going up to CEO Elon Musk with the Cybertruck’s design and Musk going: “I love it, let’s figure out how to build this.” Then Tesla’s engineers and designers had to make this thing a reality Moving to the interior design, that’s where I think that Tesla had to compromise a lot to make the exterior shape of the Cybertruck work. Instead, I now feel like the market is mostly existing Tesla owners who don’t even really need a truck, but they are getting one because Tesla is making this.', 'score': 0.69845337, 'raw_content': None}]

### 🤖 Agent’s Answer
🔍 According to a web search, reviews of the new Tesla Cybertruck highlight its impressive driving performance and distinctive design. Drivers have noted that the Cybertruck offers quick acceleration comparable to Tesla's Model S and performs well even on steep inclines. Some reviewers emphasize its utility, capable of city driving as well as off-road and hauling tasks, with a range of around 318 miles when fully charged.

However, the unusual angular design and interior have been points of mixed opinions. While the exterior is eye-catching and futuristic, some feel that the interior shows compromises due to the unique shape. Reviewers also point out that the Cybertruck may attract mostly existing Tesla fans rather than traditional truck buyers. Overall, it is praised for its strong tech and performance, though some aspects like range and design generate debate compared to competing electric trucks.

In [16]:
q2 = "Summarize the main feature in one paragraph"

print_markdown(f"**User:** {q2}")

run2 = await Runner.run(
    starting_agent = live_researcher_agent,
    input = q2,
    session = session,
)

print_markdown(f"### 🤖 Agent’s Answer\n{run2.final_output}")

**User:** Summarize the main feature in one paragraph

### 🤖 Agent’s Answer
The Tesla Cybertruck stands out with its striking, angular design and powerful performance, delivering quick acceleration comparable to the Model S and a versatile driving experience suited for both city and off-road use. It offers a range of around 318 miles on a full charge and strong towing and hauling capabilities, positioning it as a high-tech utility vehicle. While its futuristic exterior draws attention, the interior design reflects necessary compromises, appealing primarily to existing Tesla enthusiasts. Overall, it combines innovative technology, robust functionality, and a bold aesthetic that challenges traditional pickup truck norms.